In [ ]:
!pip install requests

In [58]:
import requests
from requests.auth import HTTPBasicAuth
import json
import time
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def read_lockfile():
    path = r"D:/Riot Games/League of Legends/lockfile" 
    with open(path, "r") as f:
        name, pid, port, password, protocol = f.read().split(":")
    return port, password

def lcu_request(endpoint):
    """Generic GET request to LCU"""
    port, password = read_lockfile()
    url = f"https://127.0.0.1:{port}{endpoint}"
    auth = HTTPBasicAuth("riot", password)
    resp = requests.get(url, auth=auth, verify=False)
    return resp.json()

def live_request(endpoint):
    """Generic GET request to Live Client API"""
    url = f"https://127.0.0.1:2999{endpoint}"
    resp = requests.get(url, verify=False)
    return resp.json()

In [60]:
def get_current_summoner():
    return lcu_request("/lol-summoner/v1/current-summoner")

def get_gameflow_phase():
    return lcu_request("/lol-gameflow/v1/gameflow-phase")

def get_champselect_session():
    return lcu_request("/lol-champ-select/v1/session")

def get_active_player():
    return live_request("/liveclientdata/activeplayer")

def get_player_list():
    return live_request("/liveclientdata/playerlist")

def get_game_stats():
    return live_request("/liveclientdata/gamestats")

def get_event_data():
    return live_request("/liveclientdata/eventdata")

ALL_ENDPOINTS = {
    "LCU": {
        "Current Summoner": get_current_summoner,
        "Gameflow Phase": get_gameflow_phase,
        "Champ Select Session": get_champselect_session,
    },
    "Live Client": {
        "Active Player": get_active_player,
        "Player List": get_player_list,
        "Game Stats": get_game_stats,
        "Event Data": get_event_data
    },
}

for category, endpoints in ALL_ENDPOINTS.items():
    print(f"\n==== {category} Endpoints ====")
    for name, func in endpoints.items():
        try:
            print(f"\n--- {name} ---")
            data = func()
            print(data)
        except Exception as e:
            print(f"Failed to fetch {name}: {e}")



==== LCU Endpoints ====

--- Current Summoner ---
{'accountId': 238448945, 'displayName': '', 'gameName': 'CritHappens', 'internalName': '', 'nameChangeFlag': False, 'percentCompleteForNextLevel': 74, 'privacy': 'PUBLIC', 'profileIconId': 6623, 'puuid': '24fd96a4-0a74-5c94-8438-f56d5bfa9ab6', 'rerollPoints': {'currentPoints': 500, 'maxRolls': 2, 'numberOfRolls': 2, 'pointsCostToRoll': 250, 'pointsToReroll': 0}, 'summonerId': 105732387, 'summonerLevel': 138, 'tagLine': '6969', 'unnamed': False, 'xpSinceLastLevel': 2447, 'xpUntilNextLevel': 3264}

--- Gameflow Phase ---
EndOfGame

--- Champ Select Session ---
{'errorCode': 'RPC_ERROR', 'httpStatus': 404, 'implementationDetails': {}, 'message': 'No active delegate'}

==== Live Client Endpoints ====

--- Active Player ---
Failed to fetch Active Player: HTTPSConnectionPool(host='127.0.0.1', port=2999): Max retries exceeded with url: /liveclientdata/activeplayer (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x

In [57]:
# ======== Event Watcher ========
def event_to_text(event):
    etype = event["EventName"]
    t = int(event["EventTime"])
    mm, ss = divmod(t, 60)

    if etype == "GameStart":
        desc = "The game has started!"
    elif etype == "MinionsSpawning":
        desc = "Minions have spawned."
    elif etype == "ChampionKill":
        desc = f'{event.get("KillerName","?")} killed {event.get("VictimName","?")}'
    elif etype == "TurretKilled":
        desc = f'{event.get("KillerName","?")} destroyed {event.get("TurretKilled","a turret")}'
    elif etype == "DragonKill":
        desc = f'{event.get("KillerName","?")} killed a {event.get("DragonType","dragon")} dragon'
    elif etype == "BaronKill":
        desc = f'{event.get("KillerName","?")} killed Baron Nashor'
    else:
        desc = etype

    return f"[{mm:02d}:{ss:02d}] {desc}"

def watch_events(poll_interval=1.0):
    """Continuously fetch and print new events"""
    seen_ids = set()
    while True:
        try:
            events = get_event_data()
            for e in events.get("Events", []):
                if e["EventID"] not in seen_ids:
                    seen_ids.add(e["EventID"])
                    print(event_to_text(e))
        except Exception as ex:
            print("Error fetching events:", ex)
        time.sleep(poll_interval)

# ======== Demo Runner ========
if __name__ == "__main__":
    print("\n🔴 Now watching live events... (Ctrl+C to stop)")
    watch_events()


🔴 Now watching live events... (Ctrl+C to stop)
Error fetching events: HTTPSConnectionPool(host='127.0.0.1', port=2999): Max retries exceeded with url: /liveclientdata/eventdata (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002172F60E610>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Error fetching events: HTTPSConnectionPool(host='127.0.0.1', port=2999): Max retries exceeded with url: /liveclientdata/eventdata (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002172F60E610>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
Error fetching events: HTTPSConnectionPool(host='127.0.0.1', port=2999): Max retries exceeded with url: /liveclientdata/eventdata (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002172F630AD0>: Fa

KeyboardInterrupt: 